<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datacamp.com/tutorial/mistral-7b-tutorial

# DEPENDENCIES

In [ ]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
#! pip install datasets ninja packaging --quiet
#! pip install datasets  --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
# versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.2, 0.9.3, 0.9.4)
#ERROR: No matching distribution found for trl==0.0.99
! pip install trl==0.8.6 -q

In [ ]:
import torch
import tensorboard

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [ ]:
# set device
device = 'cuda'

In [ ]:
torch.__version__

'2.3.0+cu121'

In [ ]:
#!apt-get update && apt-get install -y cuda-11-0

In [ ]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Thu Jun 13 00:58:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   53C    P8       

In [ ]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

# Data loading


https://huggingface.co/datasets/sakharamg/AviationQA

In [ ]:
#! pip install datasets  --quiet

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset Emotion")
dataset = load_dataset("dair-ai/emotion")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})


In [ ]:
train_question=dataset['text']
train_answer=dataset['label']

In [ ]:
!rm -rf /content/summary-qa.txt
filename='/content/summary-qa.txt'

import re

# python object to be appended
for n in range(10000):
    if train_answer[n] == None:
       train_answer[n] = 'Not possible to get or use'
    string = train_answer[n]
    #print(string)
    #result = string.replace("\nRef", "-Ref")
    str_question=train_question[n]
    question= re.sub("\n", "", str_question)
    answer = re.sub("\n", "", str(string))
    #print(answer)
    if len(answer)==0:
        answer='Not possible to get or use'
    text='<s>[INST] %s [/INST] %s </s>'%(question,answer)
    #print(text)
    with open(filename, 'a') as f:
        f.write(text + '\n')

In [ ]:
train_question[8]

'i have been with petronas for years i feel that petronas has performed well and made a huge profit'

label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [ ]:
train_answer[8]

1

In [ ]:
text0 = load_dataset("text", data_files="/content/summary-qa.txt", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#text0['text']

In [ ]:
nrec=10000

dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
dataset_final_text=text0[0:nrec]['text']

In [ ]:
dataset[84]

{'text': 'i feel very happy and excited since i learned so many things',
 'label': 1}

In [ ]:
text0[84]

{'text': '<s>[INST] i feel very happy and excited since i learned so many things [/INST] 1 </s>'}

In [ ]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer
datasetF['text'] = dataset_final_text

In [ ]:
datasetF

,Question,Answer,text
0,i didnt feel humiliated,0,<s>[INST] i didnt feel humiliated [/INST] 0 </s>
1,i can go from feeling so hopeless to so damned...,0,<s>[INST] i can go from feeling so hopeless to...
2,im grabbing a minute to post i feel greedy wrong,3,<s>[INST] im grabbing a minute to post i feel ...
3,i am ever feeling nostalgic about the fireplac...,2,<s>[INST] i am ever feeling nostalgic about th...
4,i am feeling grouchy,3,<s>[INST] i am feeling grouchy [/INST] 3 </s>
...,...,...,...
9995,i have a desk job and sit on my ass all day lo...,4,<s>[INST] i have a desk job and sit on my ass ...
9996,i feel like such a confused person lately sigh,4,<s>[INST] i feel like such a confused person l...
9997,i feel ive been beaten down by the words of me...,0,<s>[INST] i feel ive been beaten down by the w...
9998,i fully understand the feeling of being beaten...,0,<s>[INST] i fully understand the feeling of be...


In [ ]:
datasetF['Question'][84]

'i feel very happy and excited since i learned so many things'

In [ ]:
datasetF['Answer'][84]

1

In [ ]:
datasetF['text'][84]

'<s>[INST] i feel very happy and excited since i learned so many things [/INST] 1 </s>'

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            52Gi       2.3Gi       6.1Gi       5.0Mi        44Gi        50Gi
Swap:             0B          0B          0B


# Loading the Mistral 7B model

In [ ]:
!pip install mistral_inference -q

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install accelerate -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1" #10 june 2024,

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
#model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

Inference

Build an inference pipeline with tokenizer and the model.

In [ ]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt0="What is the capital of russia?"
#prompt="What are some interesting sites to visit in the Bay Area?"

prompt = f"Instruct: Find THE answer for the following  question.\n{prompt0}\nOutput:\n"
#prompt = f"Instruct: Find only the answer for the following  question.\n{prompt0}"

#prompt="What are some interesting sites to visit in the Bay Area?"
#prompt="What is the capital of russia?"
outputs = pipe(prompt0, max_new_tokens=128, temperature=0.9,do_sample=True,top_k=30, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt0)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt0):].strip()}")


Question: What is the capital of russia?

Generated Answer:
Moscow


In [ ]:
print(outputs[0]['generated_text'])

What is the capital of russia?
Moscow


In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
datasetF

,Question,Answer,text
0,i didnt feel humiliated,0,<s>[INST] i didnt feel humiliated [/INST] 0 </s>
1,i can go from feeling so hopeless to so damned...,0,<s>[INST] i can go from feeling so hopeless to...
2,im grabbing a minute to post i feel greedy wrong,3,<s>[INST] im grabbing a minute to post i feel ...
3,i am ever feeling nostalgic about the fireplac...,2,<s>[INST] i am ever feeling nostalgic about th...
4,i am feeling grouchy,3,<s>[INST] i am feeling grouchy [/INST] 3 </s>
...,...,...,...
9995,i have a desk job and sit on my ass all day lo...,4,<s>[INST] i have a desk job and sit on my ass ...
9996,i feel like such a confused person lately sigh,4,<s>[INST] i feel like such a confused person l...
9997,i feel ive been beaten down by the words of me...,0,<s>[INST] i feel ive been beaten down by the w...
9998,i fully understand the feeling of being beaten...,0,<s>[INST] i fully understand the feeling of be...


In [ ]:
index=2


prompt = datasetF['Question'][index]
print(prompt)

prompt = datasetF['Answer'][index]
print(prompt)

prompt = datasetF['text'][index]
print(prompt)

im grabbing a minute to post i feel greedy wrong
3
<s>[INST] im grabbing a minute to post i feel greedy wrong [/INST] 3 </s>


MODEL GENERATION - ZERO SHOT

In [ ]:
index=2

#features: ['id', 'Question', 'Answer'],
#prompt = dataset[index]['Question']
#summary = dataset[index]['Answer']

prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

original_model = model

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}"

res = original_model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=128, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)

#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to('cuda') # Assuming you want to run on GPU
#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to(torch.bfloat16)
#res = model.generate(input_ids, max_length=512, do_sample=False)
#output = tokenizer.decode(res[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)

print(f'Dialogue :\n{prompt}')
print(dash_line)

print(f'ACTION BY THE AGENT-HUMAN:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Dialogue :
im grabbing a minute to post i feel greedy wrong
---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
3

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Instruct: provide an answer for the following dialog.
im grabbing a minute to post i feel greedy wrong

Answer: It's okay to feel greedy sometimes, but it's important to remember that it's not always the right choice. It's important to consider the impact of your actions on others and to strive to make choices that are fair and ethical.


In [ ]:
print(len(datasetF))

10000


In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

# Adding the adopter to the layer

In [ ]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# Hyperparmeters

In [ ]:
!pip install accelerate -U

https://github.com/huggingface/transformers/issues/26969

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/Mistral-7B-v0.1_Emotion",

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=100,                     # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #gradient_checkpointing_kwargs={"use_reentrant": True},
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    #learning_rate=1.41e-5,
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [ ]:
#print(dataset.column_names)
print(datasetF.columns)

Index(['Question', 'Answer', 'text'], dtype='object')


In [ ]:
print(datasetF['Answer'][2])

3


# Supervised fine-tuning (SFT)  parameters

In [ ]:
!pip install trl -q

https://pytorch.org/docs/stable/checkpoint.html#torch.utils.checkpoint.checkpoint

The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

The use_reentrant=False option is generally recommended because:

1.- Performance: It is often faster, especially when there are many re-entrant functions.

2.- Safety: It avoids potential issues with re-entrant functions that might modify global states during the backward pass.

The use_reentrant=True option exists for backward compatibility and situations where modifying the code is difficult. However, it can be less efficient and potentially lead to unexpected behavior in certain cases.

In [ ]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint
#torch.utils.checkpoint.checkpoint(use_reentrant=False)

In [ ]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)

max_seq_length = 3072 # max sequence length for model and packing of the dataset

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
#model.to(device)


trainer = SFTTrainer(
    model=original_model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    #device_map={"": device},
    #formatting_func=formatting_func,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

# Model training AND Saving the fine-tuned model



```
Step	Training Loss
10	2.522100
20	2.070900
30	1.817700
40	1.629200
50	1.235400
60	0.922900
70	0.693400
80	0.460600
90	0.434600
100	0.341900
110	0.344800
120	0.285800
130	0.252200
140	0.193300
150	0.135400
160	0.098400
170	0.053000
180	0.050500
190	0.030000
200	0.034700
210	0.026400
220	0.028900
230	0.026400
240	0.023000
250	0.025500
260	0.020300
270	0.026600
280	0.017700
290	0.023700
300	0.019700
310	0.020100
320	0.020500
330	0.017500
340	0.019900
350	0.015500
360	0.020200
370	0.015800
380	0.018100
390	0.015300
400	0.016100
410	0.016300
420	0.012900
430	0.014700
440	0.011100
450	0.014900
460	0.010900
470	0.012700
480	0.011000
490	0.012400
500	0.009900
510	0.009000
520	0.010100
530	0.009000
540	0.009800
550	0.007200
560	0.007000
570	0.007100
580	0.007800
590	0.009600
600	0.006500
610	0.007300
620	0.006400
630	0.008900
640	0.007000
650	0.009400
660	0.008600
670	0.007800
680	0.007600
690	0.006600
700	0.008800
710	0.007300
720	0.009200
730	0.007500
740	0.011600
750	0.010600
760	0.010000
770	0.010400
780	0.010400
790	0.011800
800	0.010800
810	0.013900
820	0.009600
830	0.011800
840	0.011400
850	0.010300
860	0.012300
870	0.011100
880	0.012200
890	0.010500
900	0.015300
910	0.009600
920	0.012300
930	0.009300
940	0.011100
950	0.011400
960	0.009100
970	0.011700
980	0.009800
990	0.011100
```




`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(

its related to past_key_values, you can disable this warning by setting
model.config.use_cache = False, when using gradient checkpointing during training.
but during inference make sure to set it back to True.

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(

possible solution:
model.gradient_checkpointing_enable() fix it. maybe you can try @lucasjinreal    

https://github.com/huggingface/transformers/issues/28536

In [ ]:
#del device
#model.to(torch.device('cuda'))
trainer.train()

# /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py
#  1406

# save model
trainer.save_model()



```
EPOCH=40

Step	Training Loss
10	2.514300
20	2.062500
30	1.790200
40	1.603200
50	1.203800
60	0.903300
70	0.666400
80	0.452000
90	0.432500
100	0.333900
110	0.338800
120	0.276700
130	0.243900
140	0.182200
150	0.124600
160	0.090600
170	0.055700
180	0.049800
190	0.033600
200	0.037300
210	0.024600
220	0.029300
230	0.024800
240	0.025200
250	0.025000
260	0.020000
270	0.025700
280	0.017400
290	0.021400
300	0.016600
310	0.017500
320	0.017500
330	0.015700
340	0.017400
350	0.014400
360	0.016500
370	0.012900
380	0.016600
390	0.011900
400	0.012800
410	0.014100
420	0.013000
430	0.014300
440	0.013100
450	0.019100
460	0.012300
470	0.014700
480	0.105400
490	0.013400
500	0.012500
510	0.010100
520	0.012100
530	0.010900
540	0.009900
550	0.008500
560	0.009200
570	0.008200
580	0.010200
590	0.010200
600	0.010200
610	0.010600
620	0.009500
630	0.011200
640	0.008300
650	0.010800
660	0.010600
670	0.009400
680	0.007800
690	0.008100
700	0.009900
710	0.007200
720	0.010100




```




```
EPOCH=25
Step	Training Loss
10	2.503100
20	2.069500
30	1.792300
40	1.583100
50	1.190000
60	0.879000
70	0.667700
80	0.452700
90	0.428900
100	0.340300
110	0.338100
120	0.282200
130	0.243600
140	0.180700
150	0.132300
160	0.086300
170	0.051900
180	0.047300
190	0.033100
200	0.034900
210	0.026900
220	0.030600
230	0.024700
240	0.024200
250	0.024800
260	0.019700
270	0.025000
280	0.018100
290	0.021900
300	0.018100
310	0.019300
320	0.020000
330	0.017200
340	0.016400
350	0.015800
360	0.017000
370	0.013000
380	0.015500
390	0.040200
400	0.016500
410	0.020600
420	0.014900
430	0.012400
440	0.020400
450	0.011100
```


In [ ]:
# free the memory again
del model
del original_model
del trainer
torch.cuda.empty_cache()

# Model evaluation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
peft_model_id = "/content/gdrive/MyDrive/model/Mistral-7B-v0.1_Emotion"


# Load Model with PEFT adapter
modelnew = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  load_in_8bit=True # Load the model in 8-bit precision
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)


# load into pipeline
generation_pipeline = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

In [61]:
# Clear the cache to release memory
import gc
gc.collect()
torch.cuda.empty_cache()

In [62]:
index=9106

In [63]:
print(f"Question: {datasetF['Question'][index]} \nAnswer: {datasetF['Answer'][index]}")

Question: i may not feel it i m sure the wisdom that comes with age will help 
Answer: 1


In [64]:
prompt=datasetF['Question'][index]
outputs = generation_pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.1,
                              top_k=50, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [65]:
import re

response=outputs[0]['generated_text']
response= re.sub("/INST", "", response)
response= re.sub(r"\[\]", "\nSentiment with label:", response)

print(response)

#print(f"Generated Answer:\n{outputs[0]['generated_text']}")
print()
if datasetF['Answer'][index] == 0:
   print(f'Sentiment-ORIGINAL: Sadness(0)')
if datasetF['Answer'][index] == 1:
   print(f'Sentiment-ORIGINAL: Joy(1)')
if datasetF['Answer'][index] == 2:
   print(f'Sentiment-ORIGINAL: Love(2)')
if datasetF['Answer'][index] == 3:
   print(f'Sentiment-ORIGINAL: Anger(3)')
if datasetF['Answer'][index] == 4:
   print(f'Sentiment-ORIGINAL: Fear(4)')
if datasetF['Answer'][index] == 5:
   print(f'Sentiment-ORIGINAL: Surprise(5)')

i may not feel it i m sure the wisdom that comes with age will help 
Sentiment with label: 1 

Sentiment-ORIGINAL: Joy(1)
